# 12 트랜스포머의 기초

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, LayerNormalization, Dropout, Add, Input
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# CSV 파일 로드
dataframe = pd.read_csv('../data/sentiment_data.csv')

# 데이터와 라벨 추출
sentences = dataframe['sentence'].tolist()
labels = dataframe['label'].tolist()

# 임베딩 벡터 크기와 최대 문장 길이 설정
embedding_dim = 128
max_len = 10

# 토크나이저 초기화 및 텍스트를 시퀀스로 변환
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
word_index = tokenizer.word_index

# 패딩을 사용하여 시퀀스 길이를 동일하게 맞춤
data = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len, padding='post')

# 데이터셋을 훈련 세트와 검증 세트로 분리
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# 포지셔널 인코딩 함수
def get_positional_encoding(max_len, d_model):
    pos_enc = np.zeros((max_len, d_model))
    for pos in range(max_len):
        for i in range(0, d_model, 2):
            pos_enc[pos, i] = np.sin(pos / (10000 ** (2 * i / d_model)))
            if i + 1 < d_model:
                pos_enc[pos, i + 1] = np.cos(pos / (10000 ** (2 * (i + 1) / d_model)))
    return pos_enc

# 포지셔널 인코딩 생성
positional_encoding = get_positional_encoding(max_len, embedding_dim)

# 사용자 정의 레이어: MultiHeadAttention을 포함한 레이어 정의
class MultiHeadSelfAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads, key_dim):
        # 초기화 메서드에서 MultiHeadSelfAttentionLayer 클래스의 인스턴스를 초기화합니다.
        super(MultiHeadSelfAttentionLayer, self).__init__()
        # 멀티헤드 어텐션 레이어를 생성합니다.
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)
        # 레이어 정규화를 생성합니다.
        self.norm = LayerNormalization()

    def call(self, x):
        # 입력 x에 대해 멀티헤드 어텐션을 적용합니다.
        # query, value, key 모두 동일한 x를 사용합니다.
        attn_output = self.mha(query=x, value=x, key=x)
        # 잔차 연결을 적용하여 입력 x와 어텐션 출력 attn_output을 더한 후, 정규화를 적용합니다.
        attn_output = self.norm(attn_output + x)
        # 어텐션과 정규화된 출력을 반환합니다.
        return attn_output

# 모델 설정
inputs = Input(shape=(max_len,))

# 1. 임베딩 레이어: 텍스트 데이터를 임베딩 벡터로 변환합니다.
embedding_layer = Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim, input_length=max_len)
embedded_sequences = embedding_layer(inputs)

# 2. 포지셔널 인코딩 추가
embedded_sequences_with_positional_encoding = embedded_sequences + positional_encoding

# 3. 멀티헤드 어텐션 레이어 추가
attention_layer = MultiHeadSelfAttentionLayer(num_heads=8, key_dim=embedding_dim)
attention_output = attention_layer(embedded_sequences_with_positional_encoding)

# 4. 잔차 연결
attention_output_with_residual = Add()([embedded_sequences_with_positional_encoding, attention_output])

# 5. GlobalAveragePooling1D 레이어 추가
pooled_output = GlobalAveragePooling1D()(attention_output_with_residual)

# 6. 피드 포워드 네트워크
dense_layer = Dense(128, activation='relu')(pooled_output)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

# 모델 생성
model = Model(inputs=inputs, outputs=output_layer)

# 모델 컴파일
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train, np.array(y_train), epochs=10, batch_size=16, validation_data=(X_val, np.array(y_val)))

# 샘플 데이터 예측
sample_texts = ["I absolutely love this!", "I can't stand this product"]
sample_sequences = tokenizer.texts_to_sequences(sample_texts)
sample_data = tf.keras.preprocessing.sequence.pad_sequences(sample_sequences, maxlen=max_len, padding='post')
predictions = model.predict(sample_data)

for i, text in enumerate(sample_texts):
    print(f"Text: {text}")
    print(f"Prediction: {'Positive' if predictions[i] > 0.5 else 'Negative'}")


Epoch 1/10
100/100 [==============================] - 2s 10ms/step - loss: 0.6190 - accuracy: 0.6442 - val_loss: 0.0344 - val_accuracy: 0.9900
Epoch 2/10
100/100 [==============================] - 1s 9ms/step - loss: 0.0484 - accuracy: 0.9894 - val_loss: 0.0207 - val_accuracy: 0.9975
Epoch 3/10
100/100 [==============================] - 1s 9ms/step - loss: 0.0333 - accuracy: 0.9956 - val_loss: 0.0242 - val_accuracy: 0.9975
Epoch 4/10
100/100 [==============================] - 1s 8ms/step - loss: 0.0381 - accuracy: 0.9950 - val_loss: 0.0207 - val_accuracy: 0.9975
Epoch 5/10
100/100 [==============================] - 1s 9ms/step - loss: 0.0346 - accuracy: 0.9931 - val_loss: 0.0202 - val_accuracy: 0.9925
Epoch 6/10
100/100 [==============================] - 1s 9ms/step - loss: 0.0254 - accuracy: 0.9937 - val_loss: 0.0103 - val_accuracy: 0.9975
Epoch 7/10
100/100 [==============================] - 1s 9ms/step - loss: 0.0174 - accuracy: 0.9950 - val_loss: 0.0060 - val_accuracy: 0.9975
Epoch